Задание 2. Проверить, насколько хорошо работает NER
Данные брать из http://www.labinform.ru/pub/named_entities/
1. проверить NER из nltk/spacy/deeppavlov.
2. написать свой NER, попробовать разные подходы.  
a. передаём в сетку токен и его соседей.  
b. передаём в сетку только токен.  
c. свой вариант.  
3. сравнить свои реализованные подходы на качество — вывести
precision/recall/f1_score.

Загрузим библиотеки и датасет:

In [1]:
from corus import load_ne5
import nltk
import numpy as np
import pandas as pd
from razdel import tokenize
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import spacy
from spacy import displacy
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchinfo import summary

In [2]:
RANDOM_STATE = 29

In [3]:
records = load_ne5('../../Теория/Lesson_5/Collection5/')

In [4]:
nltk.download('averaged_perceptron_tagger_ru')

[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     /home/shkin/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_ru is already up-to-
[nltk_data]       date!


True

Проверим работу NER из nltk:

In [5]:
for i, rec in enumerate(records):
    tokens = nltk.word_tokenize(rec.text, language='russian')
    tagged = nltk.pos_tag(tokens, lang='rus')
    display(nltk.ne_chunk(tagged))
    if i >= 2 :
        break

К сожалению, nltk не смог сделать NER из текста на русском языке,  
ограничился только pos-tagging. Попробуем spacy. Для неё загрузим  
предобученную модель:

In [6]:
#!pip install https://huggingface.co/spacy/ru_core_news_md/resolve/main/ru_core_news_md-any-py3-none-any.whl

In [7]:
nlp = spacy.load('ru_core_news_md')

In [8]:
for i, rec in enumerate(records):
    article = nlp(rec.text)
    displacy.render(article, jupyter=True, style='ent')
    if i >= 2 :
        break

Модель spacy смогла распознать некоторые сущности.

Теперь 

Сделаем из датасета pandas датафрейм, т.к. с ним привычнее работать:

In [9]:
records = load_ne5('../../Теория/Lesson_5/Collection5/')

words_docs = []
for idx, rec in enumerate(records, 1):
    words = []
    for token in tokenize(rec.text):
        type_ent = 'OUT' # Пометим токены, не размеченные в датасете, как OUT
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break # Выходим из цикла, т.к. токен может принадлежать только одной сущности
        words.append([idx, token.text, type_ent]) # Сохраняем номер текста, токен и его сущность
    words_docs.extend(words)

In [10]:
df = pd.DataFrame(words_docs, columns=['sentence', 'word', 'tag'])
df.head()

,sentence,word,tag
0,1,Зубков,PER
1,1,вновь,OUT
2,1,вошел,OUT
3,1,в,OUT
4,1,совет,OUT


In [11]:
df['tag'].value_counts()

OUT         219214
PER          21200
ORG          13651
LOC           4568
GEOPOLIT      4356
MEDIA         2482
Name: tag, dtype: int64

Имеем большой дисбаланс классов, поэтому не будем брать  
во внимание метрику accuracy.

In [12]:
num_sentences = df['sentence'].max()
num_sentences

1000

В нашем корпусе 1000 текстов. Для разбивки трейн/тест будем  
брать тексты целиком, так как по заданию нужно кроме токена  
брать его соседей. Перемешивать будем только номера текстов,  
а не токены в них.

In [13]:
np.random.seed(RANDOM_STATE)
TEST_SIZE = 0.2

idxs = np.random.permutation(range(1, num_sentences + 1))
train_idx = idxs[:int(num_sentences*(1-TEST_SIZE))]
valid_idx = idxs[int(num_sentences*(1-TEST_SIZE)):]

X_train = df.loc[df['sentence'].isin(train_idx)]
X_valid = df.loc[df['sentence'].isin(valid_idx)]

In [14]:
X_train.head()

,sentence,word,tag
401,2,"""",OUT
402,2,Друзья,OUT
403,2,Сирии,GEOPOLIT
404,2,"""",OUT
405,2,договорились,OUT


In [15]:
X_train.shape, X_valid.shape

((211239, 3), (54232, 3))

In [16]:
y_train = X_train['tag']
y_valid = X_valid['tag']

Закодируем целевую переменную числами:

In [17]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_valid = le.transform(y_valid)

Введём понятие "окна". Это число токенов справа и слева от  
текущего токена, которые будут подаваться на вход нейросети  
вместе с центральным токеном:

In [18]:
WINDOW = 1

Создадим словарь токенов для последующего использования  
в embedding-слое:

In [19]:
dist = nltk.FreqDist(X_train['word'].values)
vocabulary = {word: count for count, word in dict(enumerate(dist.keys(), 1)).items()}
print(f'Длина словаря: {len(vocabulary)}')

Длина словаря: 30206


Не сильно большой словарь, поэтому не будем ограничивать  
его размер.

Напишем наш класс датасета, в котором опишем логику работы "окна":

In [20]:
class DataWrapper(torch.utils.data.Dataset):
    def __init__(self, data, target):
        self.data = data
        self.target = torch.from_numpy(target)
    
    def __getitem__(self, index):
        sentence = self.data.iloc[index]['sentence'] # Номер текста, к которому принадлежит токен
        length = 2 * WINDOW + 1 # Количество токенов, поступающих на вход сети, зависит от "окна"
        bow = [0] * length # Заготовка для возвращаемого вектора индексов
        idx = None
        for i in range(length):
            idx = index + i - WINDOW # Отдельно работаем с каждым токеном
            if idx >= len(self.data): # Защита от ошибки, если "окно" "уведёт" нас за пределы датасета
                idx = idx - len(self.data)
                
            # Вставляем индексы слов в заготовку. Если токен принадлежит другому тексту или токена нет
            # в словаре, то оставляем 0
            if self.data.iloc[idx]['sentence'] == sentence and self.data.iloc[idx]['word'] in vocabulary:
                bow[i] = vocabulary[self.data.iloc[idx]['word']]
        
        return torch.IntTensor(bow), self.target[index]
    
    def __len__(self):
        return len(self.data)

In [21]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

Оборачиваем датасеты в даталоадеры. Перемешивать снова не будем,  
так как всё уже сделано ранее:

In [22]:
BATCH_SIZE = 1024

torch.random.manual_seed(RANDOM_STATE)

train_dataset = DataWrapper(X_train, y_train)
train_loader = DataLoader(train_dataset, 
                          batch_size=BATCH_SIZE)

valid_dataset = DataWrapper(X_valid, y_valid)
valid_loader = DataLoader(valid_dataset, 
                          batch_size=BATCH_SIZE)

Напишем сеть:

In [23]:
class Net(nn.Module):
    def __init__(self, embedding_dim=128, n_classes=len(le.classes_)):
        super().__init__()
        self.embedding = nn.Embedding(len(vocabulary)+1, embedding_dim=embedding_dim, padding_idx=0)
        self.maxpool = nn.MaxPool1d(kernel_size=2*WINDOW+1)
        self.avgpool = nn.AvgPool1d(kernel_size=2*WINDOW+1, count_include_pad=False)
        self.fc1 = nn.Linear(embedding_dim, embedding_dim * 2)
        self.fc2 = nn.Linear(embedding_dim * 2, embedding_dim)
        self.dp = nn.Dropout(0.5)
        self.fc3 = nn.Linear(embedding_dim, n_classes)
        
        
    def forward(self, x):
        x = self.embedding(x) # Принимает индексы токенов
        x = x.permute(0, 2, 1) # Меняем местами размерности для последующих слоёв
        x = self.avgpool(x) # Своего рода flatten перед полносвязным слоем. Берем среднее значение по измерению,
                                                                            # нулевые векторы в расчет не берем
        x = torch.squeeze(x) # Убираем получившуюся после предыдущего слоя единицу в размерности
        x = self.fc1(x) # Полносвязные слои
        x = self.fc2(x)
        x = self.dp(x) # Дропаут
        x = self.fc3(x)
        
        return x
    
    # Будет возвращать метку класса для последующей оценки
    def predict(self, x):
        x = self.forward(x)
        return torch.argmax(x, dim=1)

Посмотрим на структуру получившейся сети:

In [24]:
summary(Net())

Layer (type:depth-idx)                   Param #
Net                                      --
├─Embedding: 1-1                         3,866,496
├─MaxPool1d: 1-2                         --
├─AvgPool1d: 1-3                         --
├─Linear: 1-4                            33,024
├─Linear: 1-5                            32,896
├─Dropout: 1-6                           --
├─Linear: 1-7                            774
Total params: 3,933,190
Trainable params: 3,933,190
Non-trainable params: 0

Была попытка передать лосс-функции веса классов, в результате получили  
хороший Recall за счёт существенного снижения Precision. F-мера тоже  
значительно снижалась, поэтому принято решение не указывать принудительно  
веса классов.

In [25]:
# weights = []
# for tag in le.classes_:
#     weight = len(X_train) / X_train['tag'].value_counts()[tag]
#     weights.append(weight)

In [26]:
# weights

[61.65761821366024,
 57.370722433460074,
 103.49779519843214,
 19.18610354223433,
 1.2139963104084412,
 12.36979563155121]

Напишем функцию для обучения сети. Здесь всё стандартно:

In [27]:
def train_nn(epochs=5, embedding_dim=128, return_model=False):

    torch.random.manual_seed(RANDOM_STATE)
    torch.backends.cudnn.deterministic = True

    net = Net(embedding_dim=embedding_dim).to(device)
    optimizer = torch.optim.Adam(net.parameters())
    criterion = nn.CrossEntropyLoss()#weight=weights)

    for epoch in range(epochs):
        train_losses = np.array([])
        test_losses = np.array([])
        net.train()
        
        for inputs, labels in train_loader:

            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = net(inputs)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_losses = np.append(train_losses, loss.item())

        print(f'Epoch [{epoch + 1}/{epochs}]. ' \
              f'Train Loss: {train_losses.mean():.3f}. ', end='')
        
        net.eval()
        with torch.no_grad():
            for inputs, labels in valid_loader:

                inputs, labels = inputs.to(device), labels.to(device)
                outputs = net(inputs)

                loss = criterion(outputs, labels)
                test_losses = np.append(test_losses, loss.item())

        print(f'Test loss: {test_losses.mean():.3f}.')

    print('Training is finished!')
    if return_model:
        return net

Обучим сеть с заданным ранее "окном", равным 1. То есть на вход  
сети помимо самого токена подаются ещё по одному токену с каждой  
стороны от него:

In [28]:
net = train_nn(epochs=10, return_model=True)

Epoch [1/10]. Train Loss: 0.682. Test loss: 0.539.
Epoch [2/10]. Train Loss: 0.525. Test loss: 0.466.
Epoch [3/10]. Train Loss: 0.449. Test loss: 0.417.
Epoch [4/10]. Train Loss: 0.389. Test loss: 0.383.
Epoch [5/10]. Train Loss: 0.343. Test loss: 0.360.
Epoch [6/10]. Train Loss: 0.304. Test loss: 0.345.
Epoch [7/10]. Train Loss: 0.273. Test loss: 0.335.
Epoch [8/10]. Train Loss: 0.247. Test loss: 0.330.
Epoch [9/10]. Train Loss: 0.225. Test loss: 0.328.
Epoch [10/10]. Train Loss: 0.208. Test loss: 0.327.
Training is finished!


Посмотрим на метрики:

In [29]:
def print_report(model):
    y_pred = torch.IntTensor().to(device)
    y_true = torch.IntTensor().to(device)
    for data, label in valid_loader:
        y = net.predict(data.to(device))
        y_pred = torch.cat((y_pred, y))
        y_true = torch.cat((y_true, label.to(device)))
    print(classification_report(le.inverse_transform(y_true.cpu()), le.inverse_transform(y_pred.cpu())))

In [30]:
print_report(net)

              precision    recall  f1-score   support

    GEOPOLIT       0.60      0.41      0.49       930
         LOC       0.68      0.53      0.59       886
       MEDIA       0.82      0.56      0.67       441
         ORG       0.65      0.49      0.56      2641
         OUT       0.92      0.97      0.94     45211
         PER       0.84      0.60      0.70      4123

    accuracy                           0.90     54232
   macro avg       0.75      0.59      0.66     54232
weighted avg       0.89      0.90      0.89     54232



Метрики не очень хорошие. Кроме преобладающего класса OUT,  
остальные классы плохо находятся.  
Теперь обучим сеть без "окна", т.е. на вход сети будет  
подаваться только сам токен:

In [31]:
WINDOW = 0
BATCH_SIZE = 1024

torch.random.manual_seed(RANDOM_STATE)

train_dataset = DataWrapper(X_train, y_train)
train_loader = DataLoader(train_dataset, 
                          batch_size=BATCH_SIZE)

valid_dataset = DataWrapper(X_valid, y_valid)
valid_loader = DataLoader(valid_dataset, 
                          batch_size=BATCH_SIZE)

In [32]:
net = train_nn(epochs=12, return_model=True)

Epoch [1/12]. Train Loss: 0.699. Test loss: 0.547.
Epoch [2/12]. Train Loss: 0.541. Test loss: 0.473.
Epoch [3/12]. Train Loss: 0.462. Test loss: 0.416.
Epoch [4/12]. Train Loss: 0.396. Test loss: 0.375.
Epoch [5/12]. Train Loss: 0.343. Test loss: 0.344.
Epoch [6/12]. Train Loss: 0.299. Test loss: 0.322.
Epoch [7/12]. Train Loss: 0.263. Test loss: 0.306.
Epoch [8/12]. Train Loss: 0.233. Test loss: 0.295.
Epoch [9/12]. Train Loss: 0.209. Test loss: 0.288.
Epoch [10/12]. Train Loss: 0.189. Test loss: 0.281.
Epoch [11/12]. Train Loss: 0.173. Test loss: 0.277.
Epoch [12/12]. Train Loss: 0.160. Test loss: 0.275.
Training is finished!


In [33]:
print_report(net)

              precision    recall  f1-score   support

    GEOPOLIT       0.89      0.79      0.84       930
         LOC       0.76      0.63      0.69       886
       MEDIA       0.92      0.67      0.77       441
         ORG       0.76      0.48      0.59      2641
         OUT       0.93      0.99      0.96     45211
         PER       0.96      0.58      0.72      4123

    accuracy                           0.92     54232
   macro avg       0.87      0.69      0.76     54232
weighted avg       0.92      0.92      0.92     54232



Здесь показатели уже намного выше. Recall, может быть,  
не так хорош, но Precision вполне высок.

<ins>Вывод:</ins>  
Посмотрели на работу nltk и spacy в задаче выявления именованных  
сущностей на русском тексте. Nltk не справился с задачей, так как  
не заточен на русский язык, а найти какие-то дополнительные пакеты  
для этого мне не удалось. Модель для spacy нашлась быстро, и что-то  
она находит. Далее обучили feed-forward модели для классификации  
именованных сущностей. Модель, которая получала на вход только один  
токен, без соседних, оказалась значительно лучше. Это, скорее всего,  
связано с особенностями датасета: в нём оказалось мало сущностей,  
состоящих из более, чем одного токена.